In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sf-crime/train.csv.zip
/kaggle/input/sf-crime/sampleSubmission.csv.zip
/kaggle/input/sf-crime/test.csv.zip


In [2]:

import pandas as pd
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import log_loss
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout

In [3]:
sample=pd.read_csv("/kaggle/input/sf-crime/sampleSubmission.csv.zip")
train= pd.read_csv("/kaggle/input/sf-crime/train.csv.zip")
test= pd.read_csv("/kaggle/input/sf-crime/test.csv.zip")


Step1-Convert catedorical columns to numerical representations

In [4]:

train = pd.concat([train, pd.get_dummies(train['DayOfWeek'])], axis=1)
train.drop('DayOfWeek', axis=1, inplace=True)

test = pd.concat([test, pd.get_dummies(test['DayOfWeek'])], axis=1)
test.drop('DayOfWeek', axis=1, inplace=True)
train = pd.concat([train, pd.get_dummies(train['PdDistrict'])], axis=1)
train.drop('PdDistrict', axis=1, inplace=True)

test = pd.concat([test, pd.get_dummies(test['PdDistrict'])], axis=1)
test.drop('PdDistrict', axis=1, inplace=True)
train.drop(['Address', 'Descript', 'Resolution', 'Dates'], axis=1, inplace=True)

test.drop(['Address', 'Dates'], axis=1, inplace=True)


In [5]:
test.head()

,Id,X,Y,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,BAYVIEW,CENTRAL,INGLESIDE,MISSION,NORTHERN,PARK,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN
0,0,-122.399588,37.735051,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
1,1,-122.391523,37.732432,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
2,2,-122.426002,37.792212,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
3,3,-122.437394,37.721412,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
4,4,-122.437394,37.721412,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0


In [6]:
from sklearn.model_selection import train_test_split
X=train.drop(['Category'],axis=1)

y=train['Category']
y=pd.get_dummies(y)
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25)


Using pipeline model with random forest regressor

In [7]:


from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import  log_loss


categorical_cols = [cname for cname in X_train.columns if
                    
                    X_train[cname].dtype == "object"]
numerical_cols = [cname for cname in X_train.columns if 
                X_train[cname].dtype in ['int64', 'float64']]

my_cols = categorical_cols + numerical_cols
numerical_transformer = SimpleImputer(strategy='constant')
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])



preprocessor = ColumnTransformer(transformers=[('num', numerical_transformer, numerical_cols),('cat', categorical_transformer, categorical_cols)])
model = RandomForestRegressor(n_estimators=100, random_state=0)
clf = Pipeline(steps=[('preprocessor', preprocessor),('model', model)])

clf.fit(X_train, y_train)


preds = clf.predict(test.drop('Id',axis=1))




In [8]:
from sklearn.preprocessing import LabelEncoder
LB = LabelEncoder()
tar = LB.fit_transform(train['Category'])

In [9]:
Submission = pd.DataFrame(preds, columns = LB.inverse_transform(np.linspace(0, 38, 39, dtype='int16')),index=test.index) 

Submission.to_csv('Submission.csv', index_label = 'Id')